# Baseline Soybean Classification

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from PIL import ImageFilter
import random

## File Paths

In [ ]:
train_dir = "TrainData/"
train_ann = "TrainAnnotations.csv"

In [ ]:
def load_data():
    """
    Load the image training data and classes
    :return: list of numpy arrays, list of integer labels, list of string filenames (for reference)
    """
    file_labels = pd.read_csv(train_ann)
    ann_dict = pd.Series(file_labels.annotation.values, index=file_labels.file_name).to_dict()
    image_dict = dict.fromkeys(ann_dict.keys())
    print("Loading image data into dictionary...")
    for filename in list(ann_dict.keys()):
        image_dict[filename] = np.array(Image.open(train_dir + filename))
    print("Loading complete.")
    print("Preparing training data...")
    data = []
    labels = []
    names = []
    for filename in list(ann_dict):
        data.append(image_dict[filename])
        labels.append(ann_dict[filename])
        names.append(filename)
    print("Preparation complete.")
    return data, labels, names

X, y, filenames = load_data()


## Visualize the Data

In [ ]:
def visualize_data(array):
    """
    Plot a given numpy array
    :param array: a numpy array (image)
    """
    if len(array.shape) == 3:  # has all 3 RGB channels
        plt.imshow(array)
    else:  # just a black and white image (one channel)
        plt.imshow(array, cmap='gray')
    plt.axis("off")
    plt.show()

## Feature Extraction Functions

In [ ]:
def edge_filter(array, f_type="canny"):
    """
    Detect the edges within an image using a chosen filter
    :param array: a numpy array (image)
    :param f_type: the filter type in ['canny', 'laplacian', 'sobelx', 'sobely']
    :return: a numpy array (edge image)
    """
    edges = None
    if f_type == 'canny':
        edges = cv2.Canny(array, 120, 200)
    elif f_type == 'laplacian':
        # convert to grayscale --- one channel
        # Sobel filter in both directions
        array = np.array(Image.fromarray(array).convert('L'))
        edges = np.abs(cv2.Laplacian(array, cv2.CV_64F, ksize=5))
    elif f_type == "sobelx":
        # convert to grayscale --- one channel
        # get vertical edges
        array = np.array(Image.fromarray(array).convert('L'))
        edges = np.abs(cv2.Sobel(array, cv2.CV_64F, 1, 0, ksize=5))
    elif f_type == "sobely":
        # convert to grayscale --- one channel
        # get horizontal edges
        array = np.array(Image.fromarray(array).convert('L'))
        edges = np.abs(cv2.Sobel(array, cv2.CV_64F, 0, 1, ksize=5))
    return edges

def extract_channel(array, channel='green'):
    """
    Extract one channel from the image (default green because that is arguably the most important
    for this application)
    :param array: a numpy array (image)
    :param channel: string in ('red', 'green', 'blue')
    :return: a numpy array (image, one channel)
    """
    channel_dict = {'red': 0, 'green': 1, 'blue': 2}
    matrix = array[:, :, channel_dict[channel]]
    return matrix

def extract_sift(array):
    """
    Get the SIFT features from an image
    :param array: a numpy array (image)
    :return: key points, image of the key points
    """
    sift = cv2.xfeatures2d.SIFT_create(1000)
    key_points = sift.detect(array, None)
    image = cv2.drawKeypoints(array, key_points, np.array([]), (0, 0, 255),
                              cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return key_points, image

def extract_pil_edges(array):
    """
    Get the edges from an image
    :param array: a numpy array (image)
    :return: a numpy array (edge map)
    """
    im = Image.fromarray(array)
    edge = im.filter(ImageFilter.FIND_EDGES)
    return np.array(edge)

def extract_pil_emboss(array):
    """
    Get the embossed version of an image
    :param array: a numpy array (image)
    :return: a numpy array (embossed)
    """
    im = Image.fromarray(array)
    emboss = im.filter(ImageFilter.EMBOSS)
    return np.array(emboss)

def extract_hog(array, visualize=False):
    """
    Get the Histogram of Gradients of the image
    :param array: a numpy array (image)
    :param visualize: to visualize the HOG image or not [True, False]
    :return: HOG vector (for training), HOG image (for visualization)
    """
    if visualize:
        vector, im = hog(array,
                         orientations=8,
                         pixels_per_cell=(4, 4),
                         cells_per_block=(1, 1),
                         block_norm='L2-Hys',
                         feature_vector=True,
                         visualize=True)
        return vector, im
    else:
        vector = hog(array,
                     orientations=8,
                     pixels_per_cell=(4, 4),
                     cells_per_block=(1, 1),
                     feature_vector=True,
                     block_norm='L2-Hys')
        return vector

## Features

In [ ]:
print("Extracting features...")
# lists of non-flattened numpy arrays
hogs = []
greens = []
edges = []
sifts = []
embosses = []
sift_images = []  # for visualization only
for i in range(len(X)):
    if i % 10 == 0:
        print("Preprocessing Image", i+1, '/', len(X))
    hogs.append(extract_hog(X[i], visualize=False))
    greens.append(extract_channel(X[i], channel="green"))
    edges.append(extract_pil_edges(X[i]))
    embosses.append(extract_pil_emboss(X[i]))
    sift = extract_sift(X[i])
    sifts.append(sift[0])  # the key point vector that would actually be used for data representation
    sift_images.append(sift[1])  # the image, purely for visualization of those key points
print("Feature Extraction Complete.")

## Preview Random Image

### Do any visualization you want here befoer the data is flattened in the next section

In [ ]:
# RGB
index = random.randint(0, len(X))
print("Previewing image", filenames[index], "from class", y[index])
visualize_data(X[index])

In [ ]:
# Edge
visualize_data(edges[index])

In [ ]:
# Green
visualize_data(greens[index])

In [ ]:
# Emboss
visualize_data(embosses[index])

In [ ]:
# SIFT
visualize_data(sift_images[index])

## Flatten the Data

In [ ]:
print("Flattening data...")
hogs = np.array([h.flatten() for h in hogs])
greens = [g.flatten() for g in greens]
edges = [e.flatten() for e in edges]
# sifts does not need flattened
embosses = [e.flatten() for e in embosses]
print("Data flattening complete.")

## Choose Features
### The features with which you'd like to represent each image

In [ ]:
# can be any number of features, just leave np.array([]). This allows for one feature to be used alone
# e.g. features = (np.array([]), hogs[i]); features = (np.array([]), hogs[i], greens[i])
print("Building training data...")
for i in range(len(X)):
    features = (np.array([]), hogs[i], greens[i])
    X[i] = np.concatenate(features, axis=0)
print("Building feature representation complete.")

## Split the Data

In [ ]:
X = np.array(X)
y = np.array(y)
print("Vector size:", X[0].shape)

print("Splitting the data...")
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.7,
                                                      random_state=138,
                                                      shuffle=True,
                                                      stratify=y)
print("Splitting complete.")

## Train the Model

In [ ]:
print("Training the model...")
clf = svm.SVC(kernel="rbf")
clf.fit(X_train, y_train)
print("Training complete.")

In [ ]:
# Training Accuracy
y_hat = clf.predict(X_train)
print(list(y_train))
print(list(y_hat))
accuracy = np.sum(y_train == y_hat) / y_train.shape[0]
print()
print("Training Accuracy:", accuracy)

## Validate the Model

In [ ]:
# Predict on Validation Set (validation accuracy)
print("Validating the model...")
y_hat = clf.predict(X_valid)
print()
print(list(y_valid))
print(list(y_hat))
accuracy = np.sum(y_valid == y_hat) / y_valid.shape[0]
print()
print("Validation Accuracy:", accuracy)